In [24]:
import numpy as np
# from tolsolvty import tolsolvty

# Функции

In [25]:
def write_matrix_as_intervals(filename, A):
  (m, n) = A.shape
  with open(filename, "w") as mtxfile:
    for i in range(m):
      for j in range(n):
        mtxfile.write(str(A[i][j]) + " " + str(A[i][j]) + " ")
      mtxfile.write("\n")
    mtxfile.write("\n")


def write_vector_as_intervals(filename, infb, supb):
  with open(filename, "a") as vfile:
    for i in range(len(infb)):
      vfile.write(str(infb[i]) + " " + str(supb[i]) + " ")


def write_header(filename, size):
  with open(filename, "w") as mtxfile:
    mtxfile.write(str(size) + " 1.E-9      1.     100")
    mtxfile.write("\n\n")

In [26]:
def add_ans_for_bad_matrix(filename):
  a_file = open(filename, "r")
  lines = a_file.readlines()
  a_file.close()
  with open(filename, "w") as new_file:
    dim = 0
    for line in lines:
      if "Dimension" in line:
        dim = int(line.split('=')[-1])
      elif "matrix" in line:
        for _ in range(dim):
          new_file.write('-1000000000.0, 1000000000.0\n')
      else:
        new_file.write(line)

In [27]:
def clear_columns(M):
  leftIndxs = np.arange(18)
  idx = np.argwhere(np.all(M[..., :] == 0.0, axis=0))
  M = np.delete(M, idx, axis=1)
  ind = 18
  ind = ind - len(idx)
  leftIndxs = np.delete(leftIndxs, idx)
  M = M[:ind]
  while len(idx) > 0:
    idx = np.argwhere(np.all(M[..., :] == 0.0, axis=0))
    M = np.delete(M, idx, axis=1)
    ind = ind - len(idx)
    leftIndxs = np.delete(leftIndxs, idx)
    M = M[:ind]
  return M, ind, leftIndxs


def clear_lines(M):
  return M[~(M==0).all(1)]

In [28]:
def read_inters_on_ind(indSolved, coords, filename, iteration):
  with open(filename, "r") as read_file:
    for ind_arr in indSolved[iteration]:
      for i in ind_arr:
        interval = read_file.readline().split(',')
        coords[iteration][i] += [np.array(interval).astype(np.double)]

In [29]:
def intersect(a_, b_):
  if a_ is None:
    return None
  if b_ is None:
    return None 
  if a_[0] > a_[1]:
    buf = a_[1]
    a_[1] = a_[0]
    a_[0] = buf
  if b_[0] > b_[1]:
    buf = b_[1]
    b_[1] = b_[0]
    b_[0] = buf 
  if a_[1] < b_[0] or b_[1] < a_[0]:
    return None
  return np.array([max(a_[0], b_[0]), min(a_[1], b_[1])])

# СубДифф

In [30]:
def SubDiffNewton(filename, fileout, Dim, Eps = 1.e-9, Tau = 1, IterLim = 100):
  DIM = 2*Dim 
  ip = [0 for _ in range(DIM)]
  F = [[0.0 for _ in range(DIM)] for _ in range(DIM)]
  x = [_ for _ in range(DIM)]
  xx = [0.0 for _ in range(DIM)]
  C = [[0.0 for _ in range(DIM)] for _ in range(Dim)]
  d = [0.0 for _ in range(DIM)]
  fo = open(fileout, "a")

  def solver():
    ier=0
    ip[DIM-1]=1
    for k in range(DIM-1):
      m = k
      p = abs(F[k][k])
      for i in range(k+1, DIM):
        q = abs(F[i][k])
        if p<q: 
          m=i
          p=q 
      ip[k] = m 
      p=F[m][k]
      if m != k:
        ip[DIM-1] = -ip[DIM-1] 
        F[m][k] = F[k][k] 
        F[k][k] = p 
      if p==0: 
        ier=k
        ip[DIM-1]=0 
        break
      p=1./p
      for i in range(k+1,DIM):
        F[i][k]*=-p
      for j in range(k+1,DIM):
        p=F[m][j]
        F[m][j]=F[k][j] 
        F[k][j]=p
        if p!=0: 
          for i in range(k+1,DIM): 
            F[i][j] += F[i][k] * p

    if ier!=0 or F[-1][-1]==0:
      fo.write("Sorry, the interval matrix of the equation is wrong\n")
      return 1

    for k in range(DIM-1):
      m=ip[k]
      q=xx[m]
      xx[m]=xx[k] 
      xx[k]=q
      for i in range(k+1,DIM):
        xx[i] += F[i][k]*q
    for j in range(DIM-1):
      k=DIM-j-1  
      xx[k]/=F[k][k]  
      q=-xx[k]
      for i in range(k): 
        xx[i]+=F[i][k]*q
    xx[0] /= F[0][0]    #*xx/=*F
    return 0
   

 ##################################
  fi=open(filename,"r")
  if not fi:
    fo.close()
    return


  #чтение матрицы
  for i in range(Dim):
    C[i] = list(map(float, fi.readline().split(' ')[:-1]))
    for j in range(Dim):
      u0=C[i][2*j]
      u1=C[i][2*j+1] 
      p=0.5*(u0+u1)
      if p>=0.:
        F[i][j]=p 
        F[i+Dim][j+Dim]=p
      else:
        F[i+Dim][j]=p 
        F[i][j+Dim]=p 
  fi.readline()
  #чтение вектора
  d = list(map(float, fi.readline().split(' ')[:-1]))
  for i in range(Dim):
    xx[i]=d[2*i]  
    xx[i+Dim]=d[2*i+1]
    
  fi.close()
  if solver(): 
    fo.close()
    return
  ni=0
  q = 1
  r = 1
  while r/q>Eps and ni<IterLim :
    ni+=1
    r=0  
    for i in range(DIM):
      x[i]=xx[i]
      for j in range(DIM):
        F[i][j]=0
    for i in range(Dim):
        s0=0   
        s1=0
        for j in range(Dim):
          g0=C[i][2*j]
          g1=C[i][2*j+1] 
          h0=x[j]
          h1=x[j+Dim]

          if g0*g1>0:
            l= 0 if g0>0 else 2
          else: 
            l= 1 if g0<=g1 else 3
          if h0*h1>0: 
            m= 1 if h0>0 else 3 
          else: 
            m=2 if h0<=h1 else 4

          # computing multiplication in Kaucher arithmetic,
          #                   forming the subgradient matrix F   
          switch = 4*l+m
          if switch == 1: 
            t0=g0*h0
            t1=g1*h1
            F[i][j]=g0
            F[i+Dim][j+Dim]=g1
          elif switch == 2: 
            t0=g1*h0 
            t1=g1*h1
            F[i][j]=g1 
            F[i+Dim][j+Dim]=g1
          elif switch == 3: 
            t0=g1*h0
            t1=g0*h1
            F[i][j]=g1 
            F[i+Dim][j+Dim]=g0
          elif switch == 4: 
            t0=g0*h0 
            t1=g0*h1
            F[i][j]=g0 
            F[i+Dim][j+Dim]=g0
          elif switch == 5: 
            t0=g0*h1 
            t1=g1*h1
            F[i][j+Dim]=g0 
            F[i+Dim][j+Dim]=g1
          elif switch == 6: 
            u0=g0*h1 
            v0=g1*h0  
            u1=g0*h0 
            v1=g1*h1
            if u0<v0: 
              t0=u0 
              F[i][j+Dim]=g0
            else:
              t0=v0 
              F[i][j]=g1
            if u1>v1:
              t1=u1 
              F[i+Dim][j]=g0
            else:
              t1=v1 
              F[i+Dim][j+Dim]=g1
          elif switch == 7: 
            t0=g1*h0 
            t1=g0*h0
            F[i][j]=g1 
            F[i+Dim][j]=g0
          elif switch == 8: 
            t0=0 
            t1=0
          elif switch ==  9: 
            t0=g0*h1 
            t1=g1*h0
            F[i][j+Dim]=g0 
            F[i+Dim][j]=g1
          elif switch == 10: 
            t0=g0*h1 
            t1=g0*h0
            F[i][j+Dim]=g0 
            F[i+Dim][j]=g0
          elif switch == 11: 
            t0=g1*h1 
            t1=g0*h0
            F[i][j+Dim]=g1 
            F[i+Dim][j]=g0
          elif switch == 12: 
            t0=g1*h1 
            t1=g1*h0
            F[i][j+Dim]=g1 
            F[i+Dim][j]=g1
          elif switch == 13: 
            t0=g0*h0 
            t1=g1*h0
            F[i][j]=g0 
            F[i+Dim][j]=g1
          elif switch == 14: 
            t0=0 
            t1=0
          elif switch == 15: 
            t0=g1*h1 
            t1=g0*h1
            F[i][j+Dim]=g1 
            F[i+Dim][j+Dim]=g0
          elif switch == 16: 
            u0=g0*h0 
            v0=g1*h1  
            u1=g0*h1 
            v1=g1*h0
            if u0>v0:
              t0=u0
              F[i][j]=g0
            else:
              t0=v0 
              F[i][j+Dim]=g1
            if u1<v1:
              t1=u1
              F[i+Dim][j+Dim]=g0
            else: 
              t1=v1 
              F[i+Dim][j]=g1
          s0+=t0       
          s1+=t1
        t0=s0-d[2*i]
        xx[i]=t0
        t1=s1-d[2*i+1]
        xx[i+Dim]=t1
        t0=abs(t0)
        t1=abs(t1)
        r+= t0 if t0>t1 else t1

    if solver(): 
      fo.close()
      return

    q=0 
    for i in range(DIM):
      xx[i]=x[i]-xx[i]*Tau 
      q+=abs(xx[i])

    if q==0:
      q=1
    
  fo.write("Dimension = " + str(Dim) + "\n")  
  if ni >= IterLim:
    print("Doesn't the method diverge?! \n")

  for i in range(Dim):
    u0= xx[i]  
    u1=xx[i+Dim]
    if u0 < u1:
      fo.write( str(u0) +  ", " + str(u1) + "\n")
    else:
      fo.write(str(u1) + ", " + str(u0) + "\n")
  fo.close()
  return

# work Zone

In [31]:
path_folder = ''
M1 = np.loadtxt(path_folder + 'matrix_n_phi_1.txt')
#M6 = np.loadtxt(path_folder + 'matrix_n_phi_6.txt')

In [ ]:
M1_lu = M1[0:128,0:18]
M1_ld = M1[128:, 0:18]
M1_ru = M1[0:128, 18:]
M1_rd = M1[128:, 18:]
M_128_18 = [clear_lines(M1_ld), clear_lines(M1_ru)]

In [43]:
#x = [np.array([0.5, 0.75, 1.0, 1.0, 0.75, 0.5, 0.25, 0.5, 0.75, 0.75, 0.5, 0.25, 0.0, 0.25, 0.5, 0.5, 0.25, 0.0]),
#     np.array([0.0, 0.25, 0.5, 0.5, 0.25, 0.0, 0.25, 0.5, 0.75, 0.75, 0.5, 0.25, 0.5, 0.75, 1.0, 1.0, 0.75, 0.5]),]
x = [np.random.rand(18), np.random.rand(18)]
coords = [dict(zip(range(18), [[] for _ in range(18)])), dict(zip(range(18), [[] for _ in range(18)]))]
indSolved = {0:[], 1:[]}
for mInd in range(len(M_128_18)):
  M = M_128_18[mInd]
  lines = M.shape[0]
  idxs = np.argsort([np.count_nonzero(M[i]==0) for i in range(len(M))])
  r = np.random.randint(0, 10, lines)
  b = M_128_18[0].dot(x[mInd])
  infb = b - r
  supb = b + r
  begin = 0
  mtx_num = 0
  while begin < 128:
    M_work, add, leftInd = clear_columns(M[idxs[begin:begin+18]])
    if add == 1:
      coords[mInd][leftInd[0]] += [np.array([infb[idxs[begin]], supb[idxs[begin]]])]
    else:
      indSolved[mInd] += [leftInd]
      s_mtx_num = str(mtx_num) if mtx_num >= 10 else '0' + str(mtx_num)
      filename = 'matrixpreset.txt'
      fileout = 'solutions' + str(mInd) + '.txt'
      if M_work.shape[0] != M_work.shape[1]:
        print("ALERT!")
      write_matrix_as_intervals(filename, M_work)
      write_vector_as_intervals(filename, infb[idxs[begin:begin+add]], supb[idxs[begin:begin+add]])
      SubDiffNewton(filename, fileout, add)
    begin += add
    mtx_num += 1


In [44]:
add_ans_for_bad_matrix("solutions0.txt")
read_inters_on_ind(indSolved, coords, "solutions0.txt", 0)
ans_inter = []
for i in range(18):
  coord = coords[0][i]
  res_inter = np.array([-float('infinity'), float('infinity')])
  for inter in coord:
     res_inter = intersect(res_inter, inter)
  ans_inter += [res_inter]

In [45]:
with open("answer0.txt", "w") as ansFile:
  ansFile.write('\n'.join(map(lambda invl: "["+str(invl[0])+", "+str(invl[1])+"]", ans_inter)))

In [46]:
add_ans_for_bad_matrix("solutions1.txt")
read_inters_on_ind(indSolved, coords, "solutions1.txt", 1)
ans_inter = []
for i in range(18):
  coord = coords[1][i]
  res_inter = np.array([-float('infinity'), float('infinity')])
  for inter in coord:
    res_inter = intersect(res_inter, inter)
  ans_inter += [res_inter]

In [47]:
with open("answer1.txt", "w") as ansFile:
  ansFile.write('\n'.join(map(str, ans_inter)))

# Test

In [48]:
indSolved

{0: [array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17]),
  array([ 1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17]),
  array([ 2,  3,  4,  5,  7,  8,  9, 10, 11, 14, 15, 16, 17]),
  array([ 3,  4,  5,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
  array([ 3,  4,  5,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
  array([ 5,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
  array([ 4,  5, 10, 11, 13, 14, 15, 16, 17]),
  array([10, 11, 13, 14, 15, 16, 17]),
  array([10, 11, 14, 15, 16, 17]),
  array([ 5, 10, 11, 14, 15, 16, 17]),
  array([ 5, 11, 15, 16, 17]),
  array([11, 16, 17]),
  array([11, 16, 17]),
  array([16, 17]),
  array([16, 17])],
 1: [array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17]),
  array([ 1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17]),
  array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 14, 15, 16, 17]),
  array([ 0,  1,  2,  3,  4,  5,  8,  9, 10, 11, 14, 15, 16, 17]),
  array([ 0,  

In [39]:
# SubDiffNewton('matrix00_0preset.txt', 18)

In [40]:
x

[array([0.82206141, 0.41052593, 0.04569562, 0.83575945, 0.80763117,
        0.18202781, 0.48489944, 0.25241295, 0.56828884, 0.59715959,
        0.58161962, 0.09472073, 0.18725198, 0.90252498, 0.12344984,
        0.39393135, 0.87013628, 0.80093153]),
 array([0.18226752, 0.2487451 , 0.11079757, 0.25679788, 0.38187201,
        0.05632591, 0.23797433, 0.41539752, 0.60185911, 0.08050316,
        0.09388879, 0.86595043, 0.56895582, 0.71297194, 0.25636967,
        0.92180626, 0.05408199, 0.17488849])]

In [49]:
x

[array([0.85610068, 0.12096295, 0.162609  , 0.14276898, 0.75959448,
        0.66404941, 0.67896661, 0.911243  , 0.40014403, 0.59206531,
        0.01133804, 0.01351949, 0.9852865 , 0.00196384, 0.68612229,
        0.81365491, 0.566142  , 0.44656741]),
 array([0.25228871, 0.24131221, 0.42276963, 0.72062277, 0.56978137,
        0.38908608, 0.36936801, 0.20905303, 0.75409801, 0.28342636,
        0.43686531, 0.68193439, 0.94032853, 0.90555051, 0.30569502,
        0.9831685 , 0.02526371, 0.69512677])]